# MovieLens recommendations in Keras

This notebook demonstrates how to train a Keras model for recommending movies.

It's derived originally from [an example by Maciej Kula](https://github.com/maciejkula/triplet_recommendations_keras).

In this fork I did a few additional things that aren't in the original:

* Incorporated genre metadata into the model
* Slightly modified loss function to match [BPR paper](https://arxiv.org/abs/1205.2618)
* Added a non-linear layer into the model
* Wrote log data and metadata out in TensorBoard format to enable visualization of embeddings
* Provided a function to actually query the recommendations for a given user

It's also had a few tweaks to let it run without warnings in Keras 2.

## Network architecture

From a high-level perspective, this network is a **pairwise personalized ranking model**. Given a user and two movies from the training data, it learns to give a higher score to the one which the user preferred.

This architecture is sometimes called a **triplet network**, although that terminology [originally meant](https://arxiv.org/abs/1412.6622) a model that takes three inputs of the _same_ type (e.g. three movies).

Each user, movie, and genre tag is represented by an embedding -- a vector of floats, i.e. some coordinates in a high dimensional space -- which is initialized randomly and updated as the model trains. So, over time:

* Users with similar tastes will be pushed closer to each other in user embedding space
* Movies which appeal to similar people will be pushed closer to each other in movie embedding space
* Tags which appear in similar contexts will be pushed closer together in tag embedding space

Some movies have multiple genres, e.g. _Alien_ is tagged 'Action', 'Horror', 'Sci-Fi', 'Thriller'. In these cases, the tag embeddings are simply averaged together. The final representation for a movie is its own embedding concatenated with the average of its tag embeddings.

The network's weights are shared between the positive and negative examples -- i.e. the movie and tag embedding tables. This means we learn a single embedding for each movie and each tag, not different embeddings for positive and negative context. This architecture is sometimes called a [Siamese network](https://www.quora.com/What-are-Siamese-neural-networks-what-applications-are-they-good-for-and-why).

To make predictions, we use a slightly modified version of the network, with _just one_ movie input (and a user input). This outputs a personalized ranking score for that movie, that can be used to sort that user's recommendations.

In [1]:
from __future__ import print_function

import numpy as np

from keras.models import Model
from keras.layers import Embedding, Flatten, Input, Lambda
from keras.layers.merge import concatenate, dot
from keras.layers.advanced_activations import ELU
from keras.callbacks import TensorBoard
from keras.optimizers import Adam

import data
import metrics
from net_helpers import *


def build_model(num_users, num_items, num_tags, max_tags,
                item_latent_dim, tag_latent_dim):
    
    """
    Build a model for training, and a modified model for prediction.
    You need to supply the dimensionality of the movie and genre tag
    embedding spaces; the user embedding is sized automatically to be
    equal to the sum of these, e.g. 75d movies and 25d tags will give
    you a 100d user embedding.
    
    Args:
        num_users (int): the number of users in the dataset
        num_items (int): the number of movies in the dataset
        num_tags (int): the number of distinct genre tags in the dataset
        max_tags (int): the maximum number of tags associated with a single movie
        item_latent_dim (int): the size of the movie embedding to learn
        tag_latent_dim (int): the size of the tag embedding to learn
        
    Returns:
        The training model and the prediction model
    """

    # These inputs take movie IDs for the positive and negative items
    positive_item_input = Input((1, ), name='positive_item_input')
    negative_item_input = Input((1, ), name='negative_item_input')
    
    # These take zero-padded tag ID vectors, for the positive and negative items
    positive_tags = Input((max_tags, ), name='positive_tags')
    negative_tags = Input((max_tags, ), name='negative_tags')

    # Shared embedding layer for positive and negative items
    item_embedding_layer = Embedding(
        num_items, item_latent_dim, name='item_embedding', input_length=1)
    
    # Shared embedding layer for positive and negative items' tags
    tag_embedding_layer = Embedding(
        num_tags, tag_latent_dim, name='tag_embedding', input_length=max_tags)

    # Get the embeddings corresponding to the positive and negative movie IDs;
    # Flatten just turns a matrix with one row into a vector
    
    positive_item_embedding = Flatten()(item_embedding_layer(positive_item_input))
    
    negative_item_embedding = Flatten()(item_embedding_layer(negative_item_input))
    
    # Get the embeddings for the positive and negative tags, and average them
    # together via mask_aware_mean -- see comments in net_helpers.py
    
    positive_tags_embedding = Lambda(mask_aware_mean, mask_aware_mean_output_shape,
                                     name='pos_mean')(tag_embedding_layer(positive_tags))
    
    negative_tags_embedding = Lambda(mask_aware_mean, mask_aware_mean_output_shape,
                                     name='neg_mean')(tag_embedding_layer(negative_tags))
    
    # Concatenate the movie embeddings and mean tag embeddings for the two
    # inputs, then add an Exponential Linear Unit to give the network a
    # little more flexibility
    
    positive_vec = ELU()(concatenate([positive_item_embedding, positive_tags_embedding]))
    
    negative_vec = ELU()(concatenate([negative_item_embedding, negative_tags_embedding]))
    
    # Input for the user ID
    user_input = Input((1, ), name='user_input')

    # User embedding has to have dimensionality equal to item plus tag embeddings,
    # as they need to align element-wise when calculating the scores
    user_latent_dim = item_latent_dim + tag_latent_dim
    
    # Retrieve the user embedding and add an ELU as above
    user_embedding = ELU()(Flatten()(Embedding(
        num_users, user_latent_dim, name='user_embedding', input_length=1)(
            user_input)))
    
    # Now, the final representation for each movie (positive_vec/negative_vec)
    # is an ELU-transformed concatenation of the movie embedding and the mean
    # tag embedding.
    
    # The final representation for the user is just her ELU-transformed embedding.

    # Bayesian Personalized Ranking loss (see net_helpers.py)
    loss = BprLoss(name='bpr_loss')([positive_vec, negative_vec, user_embedding])

    # Construct and compile the main model for training, with positive and
    # negative movies plus user
    
    model = Model(
        inputs=[positive_item_input, positive_tags, negative_item_input, negative_tags, user_input],
        outputs=loss)
    
    model.compile(loss=identity_loss, optimizer=Adam())
    
    # Now define a separate model for prediction, with only one movie plus user,
    # which just calculates a score by taking the dot product of the user
    # representation and the movie representation
    
    # This doesn't have any additional weights of its own, it's just a subgraph
    # of the original model but with a different output node
    
    user_dot_item = dot(
        [positive_vec, user_embedding], axes=-1, name='user_dot_item')
    
    pred_model = Model(
        inputs=[positive_item_input, positive_tags, user_input],
        outputs=user_dot_item)

    return model, pred_model

Using TensorFlow backend.


## Load and transform data
We're going to load the Movielens 100k dataset and create triplets of (user, known positive item, randomly sampled negative item).

The success metric is AUC: in this case, the probability that a randomly chosen known positive item from the test set is ranked higher for a given user than a ranomly chosen negative item.

In [2]:
item_latent_dim = 125
tag_latent_dim = 25

# Read data
train, test = data.get_movielens_data()
num_users, num_items = train.shape

item_features = data.get_movielens_item_metadata(use_item_ids=False)

max_tags = item_features.shape[1]
num_tags = item_features.max() + 1

# Prepare the test triplets
test_uid, test_pid, test_nid = data.get_triplets(test)

# Generate the simplified metadata files for TensorBoard
log_dir = '/tmp/tfboard/triplet_keras/'
items_metadata, tags_metadata = data.extract_tensorboard_metadata(log_dir)

## Build and inspect the models

Now we can build both the main training model, and the simpler prediction model, and verify their structure.

In [5]:
model, pred_model = build_model(
    num_users, num_items, num_tags, max_tags,
    item_latent_dim, tag_latent_dim)

# Print the model structure
print('Model for training:')
print(model.summary())
print()
print('Model for inference:')
print(pred_model.summary())
print()

# Sanity check, should be around 0.5
print('AUC before training %s' % metrics.full_auc(pred_model, test, item_features))

Model for training:
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
positive_item_input (InputLayer) (None, 1)             0                                            
____________________________________________________________________________________________________
positive_tags (InputLayer)       (None, 6)             0                                            
____________________________________________________________________________________________________
negative_item_input (InputLayer) (None, 1)             0                                            
____________________________________________________________________________________________________
negative_tags (InputLayer)       (None, 6)             0                                            
_______________________________________________________________________

## TODO model graphs

## Train the model

Run for a few epochs, checking the AUC after every epoch.

We'll get Keras to write out some training stats and the model weights to `log_dir` so we can explore them in [TensorBoard](https://www.tensorflow.org/get_started/summaries_and_tensorboard) afterwards.

In [6]:
tensorboard = TensorBoard(
    log_dir=log_dir,
    embeddings_freq=1,
    embeddings_layer_names=['item_embedding', 'tag_embedding', 'user_embedding'],
    embeddings_metadata={'item_embedding': items_metadata, 'tag_embedding': tags_metadata})

num_epochs = 10
checkpoint_every = 1

for epoch in range(num_epochs):

    print('Epoch %s' % epoch)

    # Sample triplets from the training data
    uid, pid, nid = data.get_triplets(train)
    ptags = item_features[pid]
    ntags = item_features[nid]

    X = {
        'user_input': uid,
        'positive_item_input': pid,
        'negative_item_input': nid,
        'positive_tags': ptags,
        'negative_tags': ntags
    }
    
    checkpoint = ((epoch + 1) % checkpoint_every == 0)
    
    if checkpoint:
        callbacks=[tensorboard]
    else:
        callbacks=[]

    model.fit(X,
              np.ones(len(uid)),
              batch_size=64,
              epochs=1,
              verbose=1,
              shuffle=True,
              callbacks=callbacks)

    if checkpoint:
        print('AUC %s' % metrics.full_auc(pred_model, test, item_features))

Epoch 0
Epoch 1/1
49906/49906 [==============================] - 7s - loss: 1.6248       

## Results

After 10 epochs, you should see an AUC of at least 0.91 above.

Now go to a terminal and run:

`tensorboard --logdir=/tmp/tfboard/triplet_keras/`

and then open this link:

http://localhost:6006/ (seems to work in Chrome only... thanks Google)

That'll let you visualize the learning curve during training, and explore the embeddings graphically.

## Querying the model

For demo purposes, let's write a function that takes a user ID, and shows you what items they've liked already, and what the model's best-bet recommendations are.

We do this by preparing a batch of user/movie/genre inputs, one per each of the _n_ movies that the user hasn't seen -- really we're making _n_ separate predictions -- then we find the top-scoring ones and return these. We're just querying one user at a time, so the user ID is repeated _n_ times in the batch.

In [65]:
import heapq

all_item_ids = np.arange(num_items, dtype=np.uint16)
names = np.array(data.get_movie_names())

def query_for_user(user_id, limit=5):
    
    # First, find out what they've already liked, so we can exclude these
    prev_liked = train.getrow(user_id).nonzero()[1]
    print('User %d has previously liked:\n' % user_id)
    print('\n'.join(np.sort(names[prev_liked])))
    
    # Build the query batch
    candidate_ids = np.setdiff1d(all_item_ids, prev_liked, assume_unique=True)
    candidate_tags = item_features[candidate_ids]
    num_candidates = len(candidate_ids)
    user_id_array = np.repeat(user_id, num_candidates)
    
    # Query the model
    predictions = pred_model.predict({
        'positive_item_input': candidate_ids,
        'positive_tags': candidate_tags,
        'user_input': user_id_array},
        batch_size=num_candidates)
    
    # Get the indices (in the candidates array) of the highest-valued outputs
    top_hits = heapq.nlargest(
        limit, xrange(num_candidates), predictions.take)
    
    # Now display them
    top_hit_ids = candidate_ids[top_hits]
    print('\nTop recommendations:\n')
    print('\n'.join(names[top_hit_ids]))

In [66]:
query_for_user(350)

User 350 has previously liked:

Back to the Future (1985)
Blade Runner (1982)
Boot, Das (1981)
Butch Cassidy and the Sundance Kid (1969)
Casablanca (1942)
Chinatown (1974)
Clockwork Orange, A (1971)
Day the Earth Stood Still, The (1951)
Empire Strikes Back, The (1980)
English Patient, The (1996)
Godfather, The (1972)
Godfather: Part II, The (1974)
Gone with the Wind (1939)
Hamlet (1996)
Henry V (1989)
Indiana Jones and the Last Crusade (1989)
It Happened One Night (1934)
Monty Python and the Holy Grail (1974)
Mr. Smith Goes to Washington (1939)
North by Northwest (1959)
Notorious (1946)
Princess Bride, The (1987)
Psycho (1960)
Rear Window (1954)
Return of the Jedi (1983)
Right Stuff, The (1983)
Silence of the Lambs, The (1991)
Star Wars (1977)
Taxi Driver (1976)
Terminator, The (1984)
To Kill a Mockingbird (1962)
Vertigo (1958)
Wild Bunch, The (1969)
Wizard of Oz, The (1939)

Top recommendations:

Alien (1979)
2001: A Space Odyssey (1968)
Aliens (1986)
Dr. Strangelove or: How I Learned